### Import statements

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
from sklearn.metrics import classification_report, confusion_matrix

from src.utils import DatasetUtils

c:\users\aakaas~1\docume~1\nyu\projects\speech~1\env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Defining system constants

In [2]:
keras = tf.keras
py.init_notebook_mode(connected=True)
utils = DatasetUtils()
encoder = utils.get_label_encoder()

### Loading data

#### Loading test data

In [3]:
test_data, test_labels = utils.get_dataset_and_encoded_labels('test_data.npy', 'test_labels.npy')
test_data_unrolled = np.load('./data/test_labels_unrolled.npy')

## Models trained on noisy data

#### Loading models and the training history

In [4]:
n_arch1 = keras.models.load_model('./models/noise_based_models/CNN.h5')
n_arch2 = keras.models.load_model('./models/noise_based_models/ResNet.h5')
n_arch3 = keras.models.load_model('./models/noise_based_models/RRNN.h5')

In [5]:
n_arch1_history = pd.read_csv('./models/noise_based_models/CNN.csv').set_index('epoch')
n_arch2_history = pd.read_csv('./models/noise_based_models/ResNet.csv').set_index('epoch')
n_arch3_history = pd.read_csv('./models/noise_based_models/RRNN.csv').set_index('epoch')

### Architectures

#### Architecture 1

In [6]:
n_arch1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 63, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 63, 48)       1200      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 63, 48)       192       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 63, 48)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 63, 48)       55344     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 63, 48)       192       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 63, 48)       0         
__________

#### Architecture 2

In [7]:
n_arch2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 63, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 63, 48)  1200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 63, 48)  192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128, 63, 48)  0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

#### Architecture 3

In [8]:
n_arch3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 63, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 63, 48)  1200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 63, 48)  192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128, 63, 48)  0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

### Evaluating the models on test data

#### Architecture 1

In [9]:
n_arch1_loss, n_arch1_acc = n_arch1.evaluate(test_data, test_labels)
print('test loss: {} - test accuracy: {}'.format(n_arch1_loss, n_arch1_acc))

6835/6835 [==============================] - 10s 1ms/step
test loss: 0.7884393064401929 - test accuracy: 0.7732260424461169


#### Architecture 2

In [10]:
n_arch2_loss, n_arch2_acc = n_arch2.evaluate(test_data, test_labels)
print('test loss: {} - test accuracy: {}'.format(n_arch2_loss, n_arch2_acc))

6835/6835 [==============================] - 13s 2ms/step
test loss: 0.6657782488961069 - test accuracy: 0.8270665691033191


#### Architecture 3

In [11]:
n_arch3_loss, n_arch3_acc = n_arch3.evaluate(test_data, test_labels)
print('test loss: {} - test accuracy: {}'.format(n_arch3_loss, n_arch3_acc))

6835/6835 [==============================] - 29s 4ms/step
test loss: 0.7539723372521545 - test accuracy: 0.8093635698610095


### Predictions

#### Obtaining predictions on test data

In [12]:
n_arch1_pred = n_arch1.predict(test_data)
n_arch2_pred = n_arch2.predict(test_data)
n_arch3_pred = n_arch3.predict(test_data)

#### Getting labels for the predicted data and test data

In [13]:
n_arch1_labels = encoder.classes_[np.argmax(n_arch1_pred, axis=-1)]
n_arch2_labels = encoder.classes_[np.argmax(n_arch2_pred, axis=-1)]
n_arch3_labels = encoder.classes_[np.argmax(n_arch3_pred, axis=-1)]

labels = encoder.classes_[np.argmax(test_labels, axis=-1)]

### Classification Reports

#### Architecture 1

In [14]:
print(classification_report(labels, n_arch1_labels))

             precision    recall  f1-score   support

       down       0.90      0.64      0.75       253
      eight       0.90      0.74      0.81       257
       five       0.84      0.77      0.81       271
       four       0.99      0.68      0.81       253
         go       0.89      0.48      0.63       251
       left       0.91      0.75      0.82       267
       nine       0.85      0.84      0.84       259
         no       0.91      0.65      0.75       252
        off       0.84      0.73      0.78       262
         on       0.90      0.82      0.86       246
        one       0.99      0.71      0.83       248
      right       0.85      0.78      0.82       259
      seven       0.96      0.78      0.86       239
        six       0.90      0.80      0.85       244
       stop       0.91      0.77      0.83       249
      three       0.90      0.43      0.58       267
        two       0.97      0.65      0.78       264
    unknown       0.59      0.97      0.73   

#### Architecture 2

In [15]:
print(classification_report(labels, n_arch2_labels))

             precision    recall  f1-score   support

       down       0.91      0.79      0.84       253
      eight       0.95      0.85      0.90       257
       five       0.78      0.90      0.84       271
       four       0.95      0.79      0.86       253
         go       0.88      0.71      0.78       251
       left       0.91      0.76      0.83       267
       nine       0.92      0.85      0.88       259
         no       0.97      0.69      0.80       252
        off       0.94      0.71      0.81       262
         on       0.93      0.84      0.88       246
        one       0.94      0.80      0.86       248
      right       0.94      0.80      0.86       259
      seven       0.94      0.90      0.92       239
        six       0.92      0.86      0.89       244
       stop       0.91      0.83      0.87       249
      three       0.88      0.54      0.67       267
        two       0.98      0.80      0.88       264
    unknown       0.67      0.96      0.79   

#### Architecture 3

In [16]:
print(classification_report(labels, n_arch3_labels))

             precision    recall  f1-score   support

       down       0.89      0.74      0.81       253
      eight       0.82      0.80      0.81       257
       five       0.77      0.79      0.78       271
       four       0.91      0.78      0.84       253
         go       0.75      0.61      0.67       251
       left       0.80      0.78      0.79       267
       nine       0.81      0.85      0.83       259
         no       0.78      0.77      0.77       252
        off       0.83      0.75      0.79       262
         on       0.85      0.79      0.82       246
        one       0.86      0.82      0.84       248
      right       0.79      0.77      0.78       259
      seven       0.86      0.85      0.86       239
        six       0.79      0.83      0.81       244
       stop       0.88      0.78      0.82       249
      three       0.78      0.67      0.72       267
        two       0.90      0.81      0.85       264
    unknown       0.76      0.91      0.82   

### Confusion matrix

#### Architecture 1

In [17]:
cm = confusion_matrix(labels, n_arch1_labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
py.iplot([go.Heatmap(z=cm, x=encoder.classes_, y=encoder.classes_)], filename='n_arch1_confusion_matrix')

#### Architecture 2

In [18]:
cm = confusion_matrix(labels, n_arch2_labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
py.iplot([go.Heatmap(z=cm, x=encoder.classes_, y=encoder.classes_)], filename='n_arch2_confusion_matrix')

#### Architecture 3

In [19]:
cm = confusion_matrix(labels, n_arch3_labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
py.iplot([go.Heatmap(z=cm, x=encoder.classes_, y=encoder.classes_)], filename='n_arch3_confusion_matrix')

### Confusion matrix for error

#### Architecture 1

In [20]:
cm = confusion_matrix(labels, n_arch1_labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
np.fill_diagonal(cm, 0)
py.iplot([go.Heatmap(z=cm, x=encoder.classes_, y=encoder.classes_)], filename='n_arch1_error_matrix')

#### Architecture 2

In [21]:
cm = confusion_matrix(labels, n_arch2_labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
np.fill_diagonal(cm, 0)
py.iplot([go.Heatmap(z=cm, x=encoder.classes_, y=encoder.classes_)], filename='n_arch2_error_matrix')

#### Architecture 3

In [22]:
cm = confusion_matrix(labels, n_arch3_labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
np.fill_diagonal(cm, 0)
py.iplot([go.Heatmap(z=cm, x=encoder.classes_, y=encoder.classes_)], filename='n_arch3_error_matrix')

### Training History

#### Architecture 1

In [23]:
py.iplot([
        go.Scatter(x=n_arch1_history.index, y=n_arch1_history.acc, mode='lines+markers', name='Training accuracy'),
        go.Scatter(x=n_arch1_history.index, y=n_arch1_history.loss, mode='lines+markers', name='Training loss'),
        go.Scatter(x=n_arch1_history.index, y=n_arch1_history.val_acc, mode='lines+markers', name='Validation accuracy'),
        go.Scatter(x=n_arch1_history.index, y=n_arch1_history.val_loss, mode='lines+markers', name='Validation loss')
    ], filename='n_arch1_history')

#### Architecture 2

In [24]:
py.iplot([
        go.Scatter(x=n_arch2_history.index, y=n_arch2_history.acc, mode='lines+markers', name='Training accuracy'),
        go.Scatter(x=n_arch2_history.index, y=n_arch2_history.loss, mode='lines+markers', name='Training loss'),
        go.Scatter(x=n_arch2_history.index, y=n_arch2_history.val_acc, mode='lines+markers', name='Validation accuracy'),
        go.Scatter(x=n_arch2_history.index, y=n_arch2_history.val_loss, mode='lines+markers', name='Validation loss')
    ], filename='n_arch2_history')

#### Architecture 3

In [25]:
py.iplot([
        go.Scatter(x=n_arch3_history.index, y=n_arch3_history.acc, mode='lines+markers', name='Training accuracy'),
        go.Scatter(x=n_arch3_history.index, y=n_arch3_history.loss, mode='lines+markers', name='Training loss'),
        go.Scatter(x=n_arch3_history.index, y=n_arch3_history.val_acc, mode='lines+markers', name='Validation accuracy'),
        go.Scatter(x=n_arch3_history.index, y=n_arch3_history.val_loss, mode='lines+markers', name='Validation loss')
    ], filename='n_arch3_history')

#### Training Accuracy

In [26]:
py.iplot([
        go.Scatter(x=n_arch1_history.index, y=n_arch1_history.acc, mode='lines+markers', name='Architecture 1'),
        go.Scatter(x=n_arch2_history.index, y=n_arch2_history.acc, mode='lines+markers', name='Architecture 2'),
        go.Scatter(x=n_arch3_history.index, y=n_arch3_history.acc, mode='lines+markers', name='Architecture 3'),
    ], filename='Training accuracy')

#### Training loss

In [27]:
py.iplot([
        go.Scatter(x=n_arch1_history.index, y=n_arch1_history.loss, mode='lines+markers', name='Architecture 1'),
        go.Scatter(x=n_arch2_history.index, y=n_arch2_history.loss, mode='lines+markers', name='Architecture 2'),
        go.Scatter(x=n_arch3_history.index, y=n_arch3_history.loss, mode='lines+markers', name='Architecture 3'),
    ], filename='Training loss')

#### Validation accuracy

In [28]:
py.iplot([
        go.Scatter(x=n_arch1_history.index, y=n_arch1_history.val_acc, mode='lines+markers', name='Architecture 1'),
        go.Scatter(x=n_arch2_history.index, y=n_arch2_history.val_acc, mode='lines+markers', name='Architecture 2'),
        go.Scatter(x=n_arch3_history.index, y=n_arch3_history.val_acc, mode='lines+markers', name='Architecture 3'),
    ], filename='Validation accuracy')

#### Validation loss

In [29]:
py.iplot([
        go.Scatter(x=n_arch1_history.index, y=n_arch1_history.val_loss, mode='lines+markers', name='Architecture 1'),
        go.Scatter(x=n_arch2_history.index, y=n_arch2_history.val_loss, mode='lines+markers', name='Architecture 2'),
        go.Scatter(x=n_arch3_history.index, y=n_arch3_history.val_loss, mode='lines+markers', name='Architecture 3'),
    ], filename='Validation loss')

## Models trained on clean data

#### Loading models and the training history

In [30]:
c_arch1 = keras.models.load_model('./models/clean_models/CNN.h5')
c_arch2 = keras.models.load_model('./models/clean_models/ResNet.h5')
c_arch3 = keras.models.load_model('./models/clean_models/RRNN.h5')

In [31]:
c_arch1_history = pd.read_csv('./models/clean_models/CNN.csv').set_index('epoch')
c_arch2_history = pd.read_csv('./models/clean_models/ResNet.csv').set_index('epoch')
c_arch3_history = pd.read_csv('./models/clean_models/RRNN.csv').set_index('epoch')

### Architectures

#### Architecture 1

In [32]:
c_arch1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 63, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 63, 48)       1200      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 63, 48)       192       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 63, 48)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 63, 48)       55344     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 63, 48)       192       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 63, 48)       0         
__________

#### Architecture 2

In [33]:
c_arch2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 63, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 63, 48)  1200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 63, 48)  192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128, 63, 48)  0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

#### Architecture 3

In [34]:
c_arch3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 63, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 63, 48)  1200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 63, 48)  192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128, 63, 48)  0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

### Evaluating the models on test data

#### Architecture 1

In [35]:
c_arch1_loss, c_arch1_acc = c_arch1.evaluate(test_data, test_labels)
print('test loss: {} - test accuracy: {}'.format(c_arch1_loss, c_arch1_acc))

6835/6835 [==============================] - 9s 1ms/step
test loss: 0.5369000360751205 - test accuracy: 0.8567666422387676


#### Architecture 2

In [36]:
c_arch2_loss, c_arch2_acc = c_arch2.evaluate(test_data, test_labels)
print('test loss: {} - test accuracy: {}'.format(c_arch2_loss, c_arch2_acc))

6835/6835 [==============================] - 13s 2ms/step
test loss: 0.5148040000461015 - test accuracy: 0.8627651792856229


#### Architecture 3

In [37]:
c_arch3_loss, c_arch3_acc = c_arch3.evaluate(test_data, test_labels)
print('test loss: {} - test accuracy: {}'.format(c_arch3_loss, c_arch3_acc))

6835/6835 [==============================] - 30s 4ms/step
test loss: 0.6793248165804885 - test accuracy: 0.8327724945309742


### Predictions

#### Obtaining predictions on test data

In [38]:
c_arch1_pred = c_arch1.predict(test_data)
c_arch2_pred = c_arch2.predict(test_data)
c_arch3_pred = c_arch3.predict(test_data)

#### Getting labels for the predicted data and test data

In [39]:
c_arch1_labels = encoder.classes_[np.argmax(c_arch1_pred, axis=-1)]
c_arch2_labels = encoder.classes_[np.argmax(c_arch2_pred, axis=-1)]
c_arch3_labels = encoder.classes_[np.argmax(c_arch3_pred, axis=-1)]

labels = encoder.classes_[np.argmax(test_labels, axis=-1)]

### Classification Reports

#### Architecture 1

In [40]:
print(classification_report(labels, c_arch1_labels))

             precision    recall  f1-score   support

       down       0.85      0.79      0.82       253
      eight       0.93      0.82      0.87       257
       five       0.88      0.86      0.87       271
       four       0.91      0.89      0.90       253
         go       0.91      0.68      0.78       251
       left       0.90      0.81      0.85       267
       nine       0.96      0.88      0.92       259
         no       0.94      0.77      0.85       252
        off       0.83      0.86      0.85       262
         on       0.94      0.88      0.91       246
        one       0.94      0.87      0.90       248
      right       0.97      0.81      0.88       259
      seven       0.93      0.89      0.91       239
        six       0.93      0.86      0.89       244
       stop       0.89      0.87      0.88       249
      three       0.79      0.63      0.70       267
        two       0.96      0.83      0.89       264
    unknown       0.74      0.96      0.84   

#### Architecture 2

In [41]:
print(classification_report(labels, c_arch2_labels))

             precision    recall  f1-score   support

       down       0.94      0.77      0.84       253
      eight       0.94      0.82      0.88       257
       five       0.93      0.84      0.88       271
       four       0.97      0.88      0.92       253
         go       0.88      0.76      0.82       251
       left       0.86      0.85      0.86       267
       nine       0.93      0.90      0.91       259
         no       0.93      0.85      0.88       252
        off       0.90      0.81      0.85       262
         on       0.95      0.86      0.90       246
        one       0.94      0.86      0.90       248
      right       0.90      0.86      0.88       259
      seven       0.96      0.89      0.92       239
        six       0.92      0.88      0.90       244
       stop       0.88      0.88      0.88       249
      three       0.87      0.64      0.74       267
        two       0.93      0.86      0.89       264
    unknown       0.74      0.95      0.83   

#### Architecture 3

In [42]:
print(classification_report(labels, c_arch3_labels))

             precision    recall  f1-score   support

       down       0.87      0.76      0.81       253
      eight       0.87      0.80      0.84       257
       five       0.85      0.80      0.83       271
       four       0.89      0.83      0.86       253
         go       0.76      0.70      0.73       251
       left       0.82      0.78      0.80       267
       nine       0.85      0.83      0.84       259
         no       0.80      0.80      0.80       252
        off       0.84      0.83      0.83       262
         on       0.83      0.80      0.81       246
        one       0.90      0.81      0.85       248
      right       0.82      0.78      0.80       259
      seven       0.86      0.87      0.87       239
        six       0.81      0.86      0.83       244
       stop       0.89      0.87      0.88       249
      three       0.76      0.72      0.74       267
        two       0.86      0.85      0.85       264
    unknown       0.81      0.91      0.86   

### Confusion matrix

#### Architecture 1

In [43]:
cm = confusion_matrix(labels, c_arch1_labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
py.iplot([go.Heatmap(z=cm, x=encoder.classes_, y=encoder.classes_)], filename='c_arch1_confusion_matrix')

#### Architecture 2

In [44]:
cm = confusion_matrix(labels, c_arch2_labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
py.iplot([go.Heatmap(z=cm, x=encoder.classes_, y=encoder.classes_)], filename='c_arch2_confusion_matrix')

#### Architecture 3

In [45]:
cm = confusion_matrix(labels, c_arch3_labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
py.iplot([go.Heatmap(z=cm, x=encoder.classes_, y=encoder.classes_)], filename='c_arch3_confusion_matrix')

### Confusion matrix for error

#### Architecture 1

In [46]:
cm = confusion_matrix(labels, c_arch1_labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
np.fill_diagonal(cm, 0)
py.iplot([go.Heatmap(z=cm, x=encoder.classes_, y=encoder.classes_)], filename='c_arch1_error_matrix')

#### Architecture 2

In [47]:
cm = confusion_matrix(labels, c_arch2_labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
np.fill_diagonal(cm, 0)
py.iplot([go.Heatmap(z=cm, x=encoder.classes_, y=encoder.classes_)], filename='c_arch2_error_matrix')

#### Architecture 3

In [48]:
cm = confusion_matrix(labels, c_arch3_labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
np.fill_diagonal(cm, 0)
py.iplot([go.Heatmap(z=cm, x=encoder.classes_, y=encoder.classes_)], filename='c_arch3_error_matrix')

### Training History

#### Architecture 1

In [49]:
py.iplot([
        go.Scatter(x=c_arch1_history.index, y=c_arch1_history.acc, mode='lines+markers', name='Training accuracy'),
        go.Scatter(x=c_arch1_history.index, y=c_arch1_history.loss, mode='lines+markers', name='Training loss'),
        go.Scatter(x=c_arch1_history.index, y=c_arch1_history.val_acc, mode='lines+markers', name='Validation accuracy'),
        go.Scatter(x=c_arch1_history.index, y=c_arch1_history.val_loss, mode='lines+markers', name='Validation loss')
    ], filename='c_arch1_history')

#### Architecture 2

In [50]:
py.iplot([
        go.Scatter(x=c_arch2_history.index, y=c_arch2_history.acc, mode='lines+markers', name='Training accuracy'),
        go.Scatter(x=c_arch2_history.index, y=c_arch2_history.loss, mode='lines+markers', name='Training loss'),
        go.Scatter(x=c_arch2_history.index, y=c_arch2_history.val_acc, mode='lines+markers', name='Validation accuracy'),
        go.Scatter(x=c_arch2_history.index, y=c_arch2_history.val_loss, mode='lines+markers', name='Validation loss')
    ], filename='c_arch2_history')

#### Architecture 3

In [51]:
py.iplot([
        go.Scatter(x=c_arch3_history.index, y=c_arch3_history.acc, mode='lines+markers', name='Training accuracy'),
        go.Scatter(x=c_arch3_history.index, y=c_arch3_history.loss, mode='lines+markers', name='Training loss'),
        go.Scatter(x=c_arch3_history.index, y=c_arch3_history.val_acc, mode='lines+markers', name='Validation accuracy'),
        go.Scatter(x=c_arch3_history.index, y=c_arch3_history.val_loss, mode='lines+markers', name='Validation loss')
    ], filename='c_arch3_history')

#### Training Accuracy

In [52]:
py.iplot([
        go.Scatter(x=c_arch1_history.index, y=c_arch1_history.acc, mode='lines+markers', name='Architecture 1'),
        go.Scatter(x=c_arch2_history.index, y=c_arch2_history.acc, mode='lines+markers', name='Architecture 2'),
        go.Scatter(x=c_arch3_history.index, y=c_arch3_history.acc, mode='lines+markers', name='Architecture 3'),
    ], filename='Training accuracy')

#### Training loss

In [53]:
py.iplot([
        go.Scatter(x=c_arch1_history.index, y=c_arch1_history.loss, mode='lines+markers', name='Architecture 1'),
        go.Scatter(x=c_arch2_history.index, y=c_arch2_history.loss, mode='lines+markers', name='Architecture 2'),
        go.Scatter(x=c_arch3_history.index, y=c_arch3_history.loss, mode='lines+markers', name='Architecture 3'),
    ], filename='Training loss')

#### Validation accuracy

In [54]:
py.iplot([
        go.Scatter(x=c_arch1_history.index, y=c_arch1_history.val_acc, mode='lines+markers', name='Architecture 1'),
        go.Scatter(x=c_arch2_history.index, y=c_arch2_history.val_acc, mode='lines+markers', name='Architecture 2'),
        go.Scatter(x=c_arch3_history.index, y=c_arch3_history.val_acc, mode='lines+markers', name='Architecture 3'),
    ], filename='Validation accuracy')

#### Validation loss

In [55]:
py.iplot([
        go.Scatter(x=c_arch1_history.index, y=c_arch1_history.val_loss, mode='lines+markers', name='Architecture 1'),
        go.Scatter(x=c_arch2_history.index, y=c_arch2_history.val_loss, mode='lines+markers', name='Architecture 2'),
        go.Scatter(x=c_arch3_history.index, y=c_arch3_history.val_loss, mode='lines+markers', name='Architecture 3'),
    ], filename='Validation loss')